In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv


In [4]:
df = pd.read_csv('/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv')

In [5]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df.drop(columns = ['RowNumber','CustomerId','Surname'],inplace=True)

In [7]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [9]:
df['Gender'].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

### One-hot Encode the Categorical Columns

In [10]:
df = pd.get_dummies(df,columns=['Geography','Gender'],drop_first=True)

In [11]:
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [12]:
X = df.drop(columns=['Exited'])
y = df['Exited'].values

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense

In [15]:
model = Sequential()

model.add(Dense(3,activation='sigmoid',input_dim=11))
model.add(Dense(1,activation='sigmoid'))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 36        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 4         
Total params: 40
Trainable params: 40
Non-trainable params: 0
_________________________________________________________________


[MLP](https://imgur.com/a/XDeHrCW)

In [17]:
model.compile(optimizer='Adam',loss='binary_crossentropy')

In [18]:
model.fit(X_train_scaled,y_train, epochs = 10)

Epoch 1/10
250/250 [==============================] - 1s 1ms/step - loss: 0.7468
Epoch 2/10
250/250 [==============================] - 0s 1ms/step - loss: 0.5846
Epoch 3/10
250/250 [==============================] - 0s 1ms/step - loss: 0.5121
Epoch 4/10
250/250 [==============================] - 0s 1ms/step - loss: 0.4803
Epoch 5/10
250/250 [==============================] - 0s 1ms/step - loss: 0.4653
Epoch 6/10
250/250 [==============================] - 0s 1ms/step - loss: 0.4566
Epoch 7/10
250/250 [==============================] - 0s 1ms/step - loss: 0.4503
Epoch 8/10
250/250 [==============================] - 0s 1ms/step - loss: 0.4451
Epoch 9/10
250/250 [==============================] - 0s 1ms/step - loss: 0.4407
Epoch 10/10
250/250 [==============================] - 0s 1ms/step - loss: 0.4370


### Finding out the Weights after Training

> ### **Last Array is the Bias**

In [19]:
model.layers[0].get_weights()

[array([[-0.4468399 ,  0.17570168, -0.03008353],
        [ 1.5558294 , -0.22702596, -1.9806209 ],
        [-0.13486326, -0.23199584,  0.12983944],
        [ 0.12591898, -0.20977072, -0.37906763],
        [-0.13095298,  0.17948566,  0.23613422],
        [-0.33643112, -0.40885535, -0.02058923],
        [-0.8354899 ,  0.29049373,  0.6455258 ],
        [ 0.23921533, -0.45500374,  0.0119699 ],
        [ 0.28753808, -0.50759876, -0.6597867 ],
        [-0.08543914,  0.37757623,  0.01993852],
        [-0.09167623,  0.05999202,  0.6671411 ]], dtype=float32),
 array([-0.5359063 , -0.26140887,  0.76342356], dtype=float32)]

In [20]:
model.layers[1].get_weights()

[array([[ 0.5859695],
        [-0.192865 ],
        [-1.5029873]], dtype=float32),
 array([-0.7092703], dtype=float32)]

### The output is not 0/1 as it is Sigmoid [0,1]

In [26]:
model.predict(X_test_scaled)

array([[0.22767714],
       [0.33063015],
       [0.17208287],
       ...,
       [0.15968227],
       [0.14102334],
       [0.18757373]], dtype=float32)

In [27]:
y_log = model.predict(X_test_scaled)

In [31]:
np.where(y_log > 0.5 , 1 , 0)

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

### Improving Accuracy
### 1.Increasing Hidden Layer
### 2.Changing *activation* = **Relu**
### 3.Increasing *Epoch Values*
### 4.Increasing *Node* in Hidden Layer

In [ ]:
model = Sequential()

model.add(Dense(11,activation='relu',input_dim=11))
model.add(Dense(11,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 11)                132       
_________________________________________________________________
dense_3 (Dense)              (None, 11)                132       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 12        
Total params: 276
Trainable params: 276
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='Adam',loss='binary_crossentropy' , metrics = ['accuracy']) #It shows Accuracy Alongside

In [ ]:
model.fit(X_train_scaled,y_train, epochs = 10 , validation_split = 0.2) #Validation split means seperating 20%

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7975

In [22]:
y_pred

array([[0.4692138],
       [0.4692138],
       [0.4692138],
       ...,
       [0.4692138],
       [0.4692138],
       [0.4692138]], dtype=float32)

In [23]:
y_pred = y_pred.argmax(axis=-1)

In [24]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7975

In [25]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

NameError: name 'history' is not defined

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])